In [1]:
# install packages not native to Google CoLab - PySpark & Java

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.<enter version>'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [5]:
### Create DataFrames to match the vine table

In [6]:
# Read in the Review dataset as a DataFrame
df.show(15)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [7]:
# Create a DataFrame for the vine_table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table_df.show(8)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 8 rows



In [8]:
# Filter DataFrame for total_votes above or equal to 20
df_1 = vine_table_df.filter(vine_table_df.total_votes >= 20).sort("total_votes")
df_1.show(25)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1RTIKR8HWSJYQ|          1|            0|         20|   N|                N|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
|R2IA8S092OPQ8D|          1|           14|         20|   N|                Y|
| RYJVJ765QAQJW|          1|           14|         20|   N|                N|
|R18YE5XIUB40P9|          5|           18|         20|   N|                Y|
|R18UIVCYVG6HKQ|          3|           13|         20|   N|                N|
|R1RA7M526HB0O7|          1|            7|         20|   N|                N|
|R30NFTKJFKHP67|          5|           20|         20|   N|                Y|
|R23Z2K6GIQ2E6O|          4|           19|         20|   N|                Y|
|R1C4DXR3JRXLL4|          1|            0|         20|   N|     

In [9]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
df_2 = df_1.filter((df_1.helpful_votes / df_1.total_votes) >= 0.5)
df_2.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26OQY0D8FB0TN|          1|           16|         20|   N|                Y|
|R152GY7O1WMJI1|          5|           19|         20|   N|                Y|
| RPILRULHOKNDQ|          5|           19|         20|   N|                N|
|R2IA8S092OPQ8D|          1|           14|         20|   N|                Y|
| R24PBSGVRTGKG|          5|           19|         20|   N|                N|
|R18YE5XIUB40P9|          5|           18|         20|   N|                Y|
|R1VC5BYOJ1XNPD|          5|           19|         20|   N|                N|
|R13TCXU89VZ0WC|          3|           19|         20|   N|                Y|
|R13Z104FLN9PIJ|          3|           13|         20|   N|                Y|
|R1Y6ZYTARUH13D|          5|           14|         20|   N|     

In [10]:
# Create paid vine DataFrame
paid_vine_df = df_2.filter(df_2.vine == 'Y')
paid_vine_df.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3R0F8ETAGTIOT|          5|           14|         20|   Y|                N|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|                N|
|R3N4QAXTQCA1ED|          5|           18|         20|   Y|                N|
|R2INH817LTJFGI|          5|           18|         20|   Y|                N|
|R369DSVA6AH6SB|          5|           16|         21|   Y|                N|
| RF41VQVEOQ4DV|          5|           17|         22|   Y|                N|
|R3QJ1YLOZNKQ4L|          5|           19|         22|   Y|                N|
| ROXZOMHBY8Y9S|          4|           21|         23|   Y|                N|
| RKKMH184GEHKD|          5|           19|         23|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|     

In [11]:
# Create unpaid vine DataFrame
unpaid_vine_df = df_2.filter(df_2.vine == 'N')
unpaid_vine_df.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26OQY0D8FB0TN|          1|           16|         20|   N|                Y|
|R152GY7O1WMJI1|          5|           19|         20|   N|                Y|
| RPILRULHOKNDQ|          5|           19|         20|   N|                N|
|R2IA8S092OPQ8D|          1|           14|         20|   N|                Y|
| R24PBSGVRTGKG|          5|           19|         20|   N|                N|
|R18YE5XIUB40P9|          5|           18|         20|   N|                Y|
|R1VC5BYOJ1XNPD|          5|           19|         20|   N|                N|
|R13TCXU89VZ0WC|          3|           19|         20|   N|                Y|
|R13Z104FLN9PIJ|          3|           13|         20|   N|                Y|
|R1Y6ZYTARUH13D|          5|           14|         20|   N|     

In [12]:
# Total number of paid reviews
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

94

In [13]:
# Total number of paid 5-star reviews
five_star_reviews_paid = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
five_star_reviews_paid

48

In [14]:
# Total Percentage of paid 5-star reviews
five_star_percent_paid = (five_star_reviews_paid / total_paid_reviews) * 100
five_star_percent_paid


51.06382978723404

In [15]:
# Total number of unpaid reviews
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews

40471

In [16]:
# Total number of unpaid 5-star reviews
five_star_reviews_unpaid = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
five_star_reviews_unpaid

15663

In [17]:
# Total Percentage of unpaid 5-star reviews
five_star_percent_unpaid = (five_star_reviews_unpaid / total_unpaid_reviews) * 100
five_star_percent_unpaid

38.701786464381904